In [8]:
import numpy as np

from manipulation import running_as_notebook

from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Box,
    DiagramBuilder,
    ModelVisualizer,
    MultibodyPlant,
    BodyIndex,
    InverseKinematics,
    RotationMatrix,
    SceneGraph,
    Simulator,
    RollPitchYaw,
    InverseDynamicsController,
    BsplineTrajectory,
    KinematicTrajectoryOptimization,
    SpatialVelocityConstraint,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    MinimumDistanceLowerBoundConstraint,
    Parser,
    PositionConstraint,
    Rgba,
    RigidTransform,
    Role,
    Solve,
    Sphere,
    StartMeshcat,
)

from manipulation import running_as_notebook
from manipulation.meshcat_utils import AddMeshcatTriad, PublishPositionTrajectory
from manipulation.utils import ConfigureParser
from centers import centers
from MIDI.midi_to_hit import midi_to_hit

from IPython.display import clear_output

# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7003


In [9]:
robot_directives = """
directives:

- add_model:
    name: iiwa_left
    file: package://drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf
    default_joint_positions:
        iiwa_joint_1: [-1]
        iiwa_joint_2: [0.1]
        iiwa_joint_3: [0]
        iiwa_joint_4: [-1.2]
        iiwa_joint_5: [0]
        iiwa_joint_6: [ 1.6]
        iiwa_joint_7: [10]
- add_weld:
    parent: world
    child: iiwa_left::iiwa_link_0
    X_PC:
        translation: [0.42, 0.35, 0]
        rotation: !Rpy { deg: [0, 0, -0]}
- add_model:
    name: drum_stick_left
    file: package://robotics_final_project/drum_sticks.sdf
- add_weld:
    parent: iiwa_left::iiwa_link_7
    child: drum_stick_left::stick
    X_PC:
        translation: [0, 0, 0.18]
        rotation: !Rpy { deg: [180, 0, 0]}

- add_model:
    name: drum_kit
    file: package://robotics_final_project/drum_kit.sdf

"""

In [10]:
stiffness=20000
dissipation=2

meshcat.Delete()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(
        builder, time_step=0.00001
    )

parser = Parser(plant)
ConfigureParser(parser)

parser.package_map().Add("robotics_final_project", "./")
parser.AddModelsFromString(robot_directives, ".dmd.yaml")
plant.Finalize()

visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    MeshcatVisualizerParams(role=Role.kIllustration),
)

collision_visualizer = MeshcatVisualizer.AddToBuilder(
    builder,
    scene_graph,
    meshcat,
    MeshcatVisualizerParams(
        prefix="collision", role=Role.kProximity, visible_by_default=False
    ),
)

kp = [100] * plant.num_positions()
ki = [1] * plant.num_positions()
kd = [20] * plant.num_positions()
iiwa_controller = builder.AddSystem(
    InverseDynamicsController(plant, kp, ki, kd, False)
)
iiwa_controller.set_name("iiwa_controller")

builder.Connect(
    plant.get_state_output_port(),
    iiwa_controller.get_input_port_estimated_state(),
)
builder.Connect(
    iiwa_controller.get_output_port_control(), plant.get_actuation_input_port()
)

diagram = builder.Build()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyMutableContextFromRoot(context)

simulator = Simulator(diagram, context)
simulator.set_target_realtime_rate(1.0)

# meshcat.StartRecording()
# simulator.AdvanceTo(5.0)

# meshcat.StopRecording() 
# meshcat.PublishRecording()

In [11]:
import numpy as np
from MIDI.midi_to_name import midi_to_name, hit_mapping
from mido import MidiFile, MetaMessage, tick2second
hits = midi_to_hit(filename = 'MIDI/midi_drums.mid')
print(hits)


[{'hit': 'ride', 'last_hit_time': 0.03750001875, 'total_time': 0.03750001875, 'strength': 82}, {'hit': 'low_tom', 'last_hit_time': 0.018055564583333333, 'total_time': 0.05555558333333334, 'strength': 57}, {'hit': 'snare', 'last_hit_time': 0.5138891458333333, 'total_time': 0.5694447291666667, 'strength': 116}, {'hit': 'ride', 'last_hit_time': 0.150000075, 'total_time': 0.7194448041666667, 'strength': 96}, {'hit': 'ride', 'last_hit_time': 0.60416696875, 'total_time': 1.3236117729166668, 'strength': 47}, {'hit': 'low_tom', 'last_hit_time': 0.018055564583333333, 'total_time': 1.3416673375000001, 'strength': 56}, {'hit': 'snare', 'last_hit_time': 0.52083359375, 'total_time': 1.8625009312500003, 'strength': 93}, {'hit': 'ride', 'last_hit_time': 0.18472231458333335, 'total_time': 2.0472232458333335, 'strength': 64}, {'hit': 'ride', 'last_hit_time': 0.616666975, 'total_time': 2.6638902208333337, 'strength': 42}, {'hit': 'mid_tom', 'last_hit_time': 0.0055555583333333335, 'total_time': 2.6694457

In [12]:
# import copy
# newhits=[]
# def timecalc(hit1, hit2):
#     halftime = 0.5 * (hit2 +hit1)
#     return halftime
# for i in range(len(hits)-1):
#     halftime = timecalc(hits[i]["total_time"], hits[i+1]["total_time"])
#     l = hits[i]
#     ll= copy.copy(l)
#     newhits.append(l)
#     # print()
#     ll["hit"] = ll["hit"] + "2"
#     ll["total_time"] = halftime
#     newhits.append(ll)
#     # print(newhits[-1]["total_time"])
# newhits

In [13]:
left = ["hi_hat", "snare", "crash", "hi_tom"]
right = ["low_tom","mid_tom", "ride"]
# left = ["hi_hat", "hi_hat2"]
# right = ["snare","snare2"]
def left_right_detector(hit):
    if hit["hit"] in left:
        side = "left"
    else:
        side = "right"
    return side

def visualize_frame(name, X_WF, length=0.15, radius=0.006):
        """
        visualize imaginary frame that are not attached to existing bodies

        Input:
            name: the name of the frame (str)
            X_WF: a RigidTransform to from frame F to world.

        Frames whose names already exist will be overwritten by the new frame
        """
        AddMeshcatTriad(
            meshcat, "painter/" + name, length=length, radius=radius, X_PT=X_WF
        )

In [14]:
# def trajopt_shelves_demo(plant, avoid_collisions=True):
# drum_stick_right = plant.GetModelInstanceByName("drum_stick_right")
drum_stick_left = plant.GetModelInstanceByName("drum_stick_left")
# right_frame = plant.GetFrameByName("stick",drum_stick_right )
left_frame = plant.GetFrameByName("stick", drum_stick_left)
avoid_collisions  = True 
for hit in hits:

    trajopt = KinematicTrajectoryOptimization(plant.num_positions(), 10)
    prog = trajopt.get_mutable_prog()
    trajopt.AddDurationCost(1.0)
    trajopt.AddPathLengthCost(1.0)
    trajopt.AddPositionBounds(
        plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()
    )
    trajopt.AddVelocityBounds(
        plant.GetVelocityLowerLimits(), plant.GetVelocityUpperLimits()
    )
    plant_context = plant.GetMyMutableContextFromRoot(context)
    translation = np.array(centers[hit["hit"]])[:3]
    rotation = RotationMatrix(RollPitchYaw(np.array(centers[hit["hit"]])[3:]))
    X_WD = RigidTransform(rotation, translation)
    visualize_frame('object',X_WD )
    num_q = plant.num_positions()
    q0 = plant.GetPositions(plant_context)

    # start constraint
    # start_constraint = PositionConstraint(
    #     plant,
    #     plant.world_frame(),
    #     X_WStart.translation(),
    #     X_WStart.translation(),
    #     gripper_frame,
    #     [0, 0.1, 0],
    #     plant_context,
    # )
    # trajopt.AddPathPositionConstraint(start_constraint, 0)
    prog.AddQuadraticErrorCost(
        np.eye(num_q), q0, trajopt.control_points()[:, 0]
    )

    
    # if left_right_detector(hit) == "right":
    #     gripper_frame =  right_frame
    # else:
    #     gripper_frame =  left_frame
    gripper_frame = left_frame
    # goal constraint
    goal_constraint = PositionConstraint(
        plant,
        plant.world_frame(),
        X_WD.translation() + np.array([0,0,0.02]),
        X_WD.translation() + np.array([0,0,0.02]),
        gripper_frame,
        [0, 0.1, 0],
        plant_context,
    )

    trajopt.AddPathPositionConstraint(goal_constraint, 1)
    prog.AddQuadraticErrorCost(
        np.eye(num_q), q0, trajopt.control_points()[:, -1]
    )

    # # start and end with zero velocity
    # trajopt.AddPathVelocityConstraint(
    #     np.zeros((num_q, 1)), np.zeros((num_q, 1)), 0
    # )
    # trajopt.AddPathVelocityConstraint(
    #     np.zeros((num_q, 1)), np.zeros((num_q, 1)), 1
    # )

    # Solve once without the collisions and set that as the initial guess for
    # the version with collisions.
    result = Solve(prog)

    if not result.is_success():
        print("Trajectory optimization failed, even without collisions!")
        print(result.get_solver_id().name())
    trajopt.SetInitialGuess(trajopt.ReconstructTrajectory(result))

    if avoid_collisions:
        # collision constraints
        collision_constraint = MinimumDistanceLowerBoundConstraint(
            plant, 0.01, plant_context, None, 0.1
        )
        evaluate_at_s = np.linspace(0, 1, 25)
        for s in evaluate_at_s:
            trajopt.AddPathPositionConstraint(collision_constraint, s)
        result = Solve(prog)
        if not result.is_success():
            print("Trajectory optimization failed")
            print(result.get_solver_id().name())

    PublishPositionTrajectory(
        trajopt.ReconstructTrajectory(result), context, plant, visualizer
    )
    collision_visualizer.ForcedPublish(
        collision_visualizer.GetMyContextFromRoot(context)
    )

    #Step 2 in process
    trajopt = KinematicTrajectoryOptimization(plant.num_positions(), 10)
    prog = trajopt.get_mutable_prog()
    trajopt.AddDurationCost(1.0)
    trajopt.AddPathLengthCost(1.0)
    trajopt.AddPositionBounds(
        plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()
    )
    trajopt.AddVelocityBounds(
        plant.GetVelocityLowerLimits(), plant.GetVelocityUpperLimits()
    )
    plant_context = plant.GetMyMutableContextFromRoot(context)

    prog.AddQuadraticErrorCost(
        np.eye(num_q), q0, trajopt.control_points()[:, 0]
    )

    # goal constraint
    goal_constraint = PositionConstraint(
        plant,
        plant.world_frame(),
        X_WD.translation() ,
        X_WD.translation() ,
        gripper_frame,
        [0, 0.1, 0],
        plant_context,
    )
    obj_vel_at_catch = .01*hit['strength']
    # if left_right_detector(hit) == "right":
    #     plant_auto_diff = plant.ToAutoDiffXd() # make sure to call this after calling plant.finalize
    #     final_vel_constraint = SpatialVelocityConstraint(
    #             plant_auto_diff,
    #             plant_auto_diff.world_frame(),
    #             obj_vel_at_catch,  # upper limit
    #             obj_vel_at_catch,  # lower limit
    #             plant_auto_diff.GetFrameByName("stick",drum_stick_right),
    #             np.array([0, 0, 0]).reshape(-1,1),
    #             plant_auto_diff.CreateDefaultContext(),
    #         )
    # else:
    #     plant_auto_diff = plant.ToAutoDiffXd() # make sure to call this after calling plant.finalize
    #     final_vel_constraint = SpatialVelocityConstraint(
    #             plant_auto_diff,
    #             plant_auto_diff.world_frame(),
    #             obj_vel_at_catch,  # upper limit
    #             obj_vel_at_catch,  # lower limit
    #             plant_auto_diff.GetFrameByName("stick",drum_stick_left),
    #             np.array([0, 0, 0]).reshape(-1,1),
    #             plant_auto_diff.CreateDefaultContext(),
    #         )    
    # trajopt.AddVelocityConstraintAtNormalizedTime(final_vel_constraint, 1)

    result = Solve(prog)

    PublishPositionTrajectory(
        trajopt.ReconstructTrajectory(result), context, plant, visualizer
    )
    collision_visualizer.ForcedPublish(
        collision_visualizer.GetMyContextFromRoot(context)
    )

Trajectory optimization failed
SNOPT
Trajectory optimization failed
SNOPT
Trajectory optimization failed
SNOPT
Trajectory optimization failed
SNOPT


KeyboardInterrupt: 